In [89]:
import os
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets

SPOTIPY_CLIENT_ID = ''
SPOTIPY_CLIENT_SECRET = ''
SPOTIPY_REDIRECT_URI = ''

In [160]:
username = 'hp5s41fd8dkrl0mi77gwptmco'
scope = 'playlist-modify-public'

token = util.prompt_for_user_token(username,scope,
                                   client_id= SPOTIPY_CLIENT_ID,
                                   client_secret = SPOTIPY_CLIENT_SECRET,
                                   redirect_uri = SPOTIPY_REDIRECT_URI)
sp = spotipy.Spotify(auth=token)

In [91]:
user_playlists = sp.current_user_playlists()['items']
user_playlists_list = []
user_playlists_dict = {}

for i in range(0,len(user_playlists)):
    user_playlists_list.append(user_playlists[i]['name'])
    user_playlists_dict[user_playlists[i]['name']] = user_playlists[i]['id']

In [103]:
user_playlists_list.sort()

# selecting 'tempo' will actually select a version of 'tempo' that is rounded to the nearest multiple of 5
audio_feature_dict = {'danceability':'danceability',
                      'energy':'energy',
                      'key':'key',
                      'loudness':'loudness',
                      'mode':'mode',
                      'speechiness':'speechiness',
                      'acousticness':'acousticness',
                      'instrumentalness':'instrumentalness',
                      'liveness':'liveness',
                      'valence':'valence',
                      'tempo':'tempo_rounded'}
audio_feature_list.sort()

radio_list = [('ascending',True),('descending',False)]
playlist_layout = widgets.Layout(margin='0px 0px 10px 200px',
                                 padding='0px 0px 0px 100px')
tiers_layout = widgets.Layout(margin='0px 75px 5px 150px')

playlist_dropdown = widgets.Dropdown(options=user_playlists_list,
                                     layout=playlist_layout)
primary_dropdown = widgets.Dropdown(options=audio_feature_dict,
                                    value='tempo_rounded',
                                    layout=tiers_layout)
secondary_dropdown = widgets.Dropdown(options=audio_feature_dict,
                                      value='danceability',
                                      layout=tiers_layout)
tertiary_dropdown = widgets.Dropdown(options=audio_feature_dict,
                                     value='valence',
                                     layout=tiers_layout)
primary_radio = widgets.RadioButtons(options=radio_list,
                                     description='Sort by: ')
secondary_radio = widgets.RadioButtons(options=radio_list,
                                       description='Sort by: ')
tertiary_radio = widgets.RadioButtons(options=radio_list,
                                      description='Sort by: ',)

playlist_selection = widgets.VBox([widgets.Label('Select the playlist to homogenize: ',layout=playlist_layout),playlist_dropdown,
                                   widgets.Label('Select which audio feature to perform the primary sort on: ',layout=tiers_layout),widgets.HBox([primary_dropdown,primary_radio]),
                                   widgets.Label('Select which audio feature to perform the secondary sort on: ',layout=tiers_layout),widgets.HBox([secondary_dropdown,secondary_radio]),
                                   widgets.Label('Select which audio feature to perform the tertiary sort on: ',layout=tiers_layout),widgets.HBox([tertiary_dropdown,tertiary_radio])])
display(playlist_selection)

In [161]:
playlist = sp.user_playlist(username, playlist_id=user_playlists_dict.get(playlist_dropdown.value))
tracks = playlist['tracks']
songs = tracks['items']

track_ids = []
track_names = []

for idx,item in enumerate(songs):
    if songs[idx]['track']['id'] != None:
        track_ids.append(songs[idx]['track']['id'])
        track_names.append(songs[idx]['track']['name'])

In [162]:
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []

audio_features = sp.audio_features(tracks=track_ids)

for idx,item in enumerate(audio_features):
    danceability.append(audio_features[idx]['danceability'])
    energy.append(audio_features[idx]['energy'])
    key.append(audio_features[idx]['key'])
    loudness.append(audio_features[idx]['loudness'])
    mode.append(audio_features[idx]['mode'])
    speechiness.append(audio_features[idx]['speechiness'])
    acousticness.append(audio_features[idx]['acousticness'])
    instrumentalness.append(audio_features[idx]['instrumentalness'])
    liveness.append(audio_features[idx]['liveness'])
    valence.append(audio_features[idx]['valence'])
    tempo.append(audio_features[idx]['tempo'])


In [163]:
data = list(zip(track_ids,track_names,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,
               liveness,valence,tempo))
playlist_df = pd.DataFrame(data, 
                           columns=['id','track_name','danceability','energy','key','loudness','mode',
                                    'speechiness','acousticness','instrumentalness','liveness','valence',
                                    'tempo'])

In [164]:
#playlist_df = playlist_df.set_index('id')
playlist_df['tempo_rounded'] = playlist_df['tempo'].apply(lambda x: 5*round(x/5))
playlist_df = playlist_df.sort_values(by=[primary_dropdown.value,secondary_dropdown.value,tertiary_dropdown.value],
                                      ascending=[primary_radio.value,secondary_radio.value,tertiary_radio.value])

In [165]:
playlist_df.head(10)

,id,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,tempo_rounded
61,7luOqEkajQajviNgwrLs60,SUN,0.559,0.436,6,-14.060,0,0.4490,0.07530,0.000000,0.0694,0.641,202.726,205
16,0gyxrfGl3szCRTQNONwgFf,Spaceship (feat. Buddy),0.456,0.729,6,-6.233,1,0.1900,0.24500,0.000000,0.1560,0.703,178.226,180
86,6Rvznyemwz5QNRao5HJq1o,SUGAR HILL,0.579,0.739,5,-8.842,0,0.0934,0.01790,0.000000,0.0446,0.912,174.324,175
49,52y9HSI3f2ajMYMCfGFS27,White Rabbit,0.456,0.521,11,-9.395,0,0.4490,0.31900,0.000430,0.2210,0.387,171.894,170
44,2dyACKRr8pnr8oLNnhgsUf,Beige (feat. Arin Ray & Elena Pinderhughes),0.773,0.578,8,-10.109,1,0.0866,0.29700,0.009170,0.0607,0.556,170.039,170
73,7qAtl4mH0HzajgYc2MOG77,Come & Stay,0.712,0.732,6,-6.090,0,0.1070,0.26500,0.502000,0.1900,0.511,163.961,165
18,50ID9KN4y15DZIqjIIJXkP,Chicago Bae,0.637,0.755,1,-6.216,1,0.1280,0.34800,0.000000,0.1280,0.667,161.024,160
15,76bdVgNUYJaozJvtKhgNd3,Hibiscus (feat. Gaby Duran & CJ Smith),0.609,0.642,0,-8.346,1,0.1940,0.64800,0.000036,0.0984,0.442,149.852,150
68,21MsEr1XKrvmZOKMorUz8v,Vibrations,0.649,0.710,5,-8.153,0,0.0543,0.00698,0.752000,0.3800,0.672,150.048,150
5,5228a1Wi1SQVVLLwK8vbEV,She Think,0.882,0.735,1,-5.372,1,0.1150,0.15300,0.000015,0.1030,0.723,149.949,150


In [166]:
playlist_stats_df = playlist_df.describe()

average_danceability = average_danceability
average_energy = average_energy
average_valence = playlist_stats_df['valence'].loc['mean'].round(2)

print('Your playlist is...')
if average_danceability >= 0 and average_danceability <= 0.25:
    print('\tReally hard to dance to! The average danceability score is:',average_danceability)

elif average_danceability > 0.25 and average_danceability <= 0.50:
    print('\tPretty hard to dance to! The average danceability score is:',average_danceability)

elif average_danceability > 0.5 and average_danceability <= 0.75:
    print('\tPretty easy to dance to! The average danceability score is:',average_danceability)

else:
    print('\tA straight rug-cutter! The average danceability score is:',average_danceability)
    
if  average_energy >= 0 and average_energy <= 0.25:
    print('\tVery low energy! The average energy score is:',average_energy)

elif average_energy > 0.25 and average_energy <= 0.5:
    print('\tPretty lethargic! The average energy score is:',average_energy)

elif average_energy > 0.5 and average_energy <= 0.75:
    print("\tGroovin'! The average energy score is:",average_energy)

else:
    print('\tElectric! The average energy score is:',average_energy)
    
if average_valence >= 0 and average_valence <= 0.25:
    print('\tExtremely melancholy... where are the tissues at?! The average valence score is:',average_valence)
    
elif average_valence > 0.25 and average_valence <= 0.5:
    print('\tKinda depressing! The average valence score is:',average_valence)
    
elif average_valence > 0.5 and average_valence <= 0.75:
    print('\tPositive and feel good! The average valence score is:',average_valence)
    
else:
    print('\tEuphoric! The average valence score is:',average_valence)

Your playlist is...
	Pretty easy to dance to! The average danceability score is: 0.67
	Groovin'! The average energy score is: 0.64
	Positive and feel good! The average valence score is: 0.6


In [133]:
sp.user_playlist_replace_tracks(user=username,
                                playlist_id=user_playlists_dict.get(playlist_dropdown.value),
                                tracks=playlist_df['id'].tolist())

{'snapshot_id': 'NCxlNzI5NDEzYWQ1MTJjMTlhNTU4MjgwMTNiYzQwMzUxZWVlODgzNjVk'}